In [3]:
#เทรนนิ่งโมเดล
import boto3
import pandas as pd
import s3fs
import sagemaker
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sagemaker.estimator import Estimator

# กำหนด role และ session
role = get_execution_role()
sess = sagemaker.Session()

# ใช้ path S3 ที่มีข้อมูล clean แล้วโดยตรง
train_data_path = 's3://stress-prediction-project/preprocessed_data_training_xgboost/preprocessed-data-training.csv'

# สร้าง estimator ของ XGBoost built-in algorithm
xgb_container = sagemaker.image_uris.retrieve("xgboost", sess.boto_region_name, version="1.5-1")

xgb = Estimator(
    image_uri=xgb_container,
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path='s3://stress-prediction-project/processed-data-training-xgboost/',
    sagemaker_session=sess
)

# กำหนด hyperparameters
xgb.set_hyperparameters(
    objective='multi:softmax',
    num_class=5,  # ปรับเป็น 10 เพราะ label อยู่ในช่วง 0–9
    eval_metric='merror',
    num_round=100
)

# เริ่มเทรนโมเดล โดยชี้ไปที่ path ข้อมูล S3
xgb.fit({'train': TrainingInput(train_data_path, content_type='csv')})

[05/28/25 06:03:11] INFO     Ignoring unnecessary instance type: None.                            ]8;id=304157;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=778084;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=796918;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=249177;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=737061;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=895515;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2025-05-28-06-03-11-137                                             

2025-05-28 06:03:11 Starting - Starting the training job......

In [4]:
#สร้าง endpoint ไว้ deployed 

from sagemaker import Session
from sagemaker.model import Model
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

# ตำแหน่งไฟล์โมเดลใน S3
model_data = 's3://stress-prediction-project/processed-data-training-xgboost/sagemaker-xgboost-2025-05-28-05-59-59-167/output/model.tar.gz'

# ดึง container image ของ XGBoost version ที่ใช้ตอนเทรน (ปรับ version ให้ตรงกับของคุณ)
xgb_container = sagemaker.image_uris.retrieve('xgboost', sess.boto_region_name, version='1.5-1')

# สร้าง Model
xgb_model = Model(
    image_uri=xgb_container,
    model_data=model_data,
    role=role,
    sagemaker_session=sess
)

# ตั้งชื่อ endpoint ที่ต้องการ
endpoint_name = 'xgboost-endpoint'

# Deploy endpoint
predictor = xgb_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge',
    endpoint_name=endpoint_name
)

print(f"Deployed endpoint: {endpoint_name}")

[05/28/25 06:05:04] INFO     Ignoring unnecessary instance type: None.                            ]8;id=271824;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=186058;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

                    INFO     Creating model with name: sagemaker-xgboost-2025-05-28-06-05-04-712    ]8;id=201432;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=288939;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[05/28/25 06:05:05] INFO     Creating endpoint-config with name xgboost-endpoint                    ]8;id=843199;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=920444;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#6019\6019]8;;\

                    INFO     Creating endpoint with name xgboost-endpoint                           ]8;id=59143;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=600382;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4841\4841]8;;\

--------!Deployed endpoint: xgboost-endpoint
